In [ ]:
import os
import random
import shutil
import zipfile
from pathlib import Path
import yaml

In [ ]:
!pip install ultralytics > /dev/null

## Pure Train (not fine tuning)

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.yaml")  # build a new model from YAML
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11n.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

In [ ]:
!pwd

In [ ]:
# Train the model
model.train(data="/content/combined_datasets/adult_child.yaml", epochs=80, imgsz=640)


In [ ]:
best_model = YOLO("runs/detect/train2/weights/best.pt")

In [ ]:
from IPython.display import Image

Image(filename="runs/detect/train2/results.png", width=800)

In [ ]:
# results = best_model.predict(source="screenshot.png", save=True)

In [ ]:
results = best_model.predict(source="/content/dataset_split/images/test/", save=True)

## finetuning

In [ ]:
# 全体から抽出したfinetuning用データセット(finetune_base_subset.zip)と新たなデータセットをcombineする

In [ ]:
def merge_and_split_finetune_data(
    base_zip: str,
    new_zip: str,
    output_root: str = "for_finetune_data",
    split_ratio: float = 0.8,
    seed: int = 42,
):
    random.seed(seed)

    tmp_dir = Path("tmp_merge")
    merged_dir = Path(output_root)
    images_dir = merged_dir / "images"
    labels_dir = merged_dir / "labels"

    # 初期化
    if tmp_dir.exists():
        shutil.rmtree(tmp_dir)
    if merged_dir.exists():
        shutil.rmtree(merged_dir)

    images_dir.mkdir(parents=True)
    labels_dir.mkdir(parents=True)

    def extract_and_copy(zip_path):
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            extract_dir = tmp_dir / Path(zip_path).stem
            zip_ref.extractall(extract_dir)

            for subdir in extract_dir.rglob("images"):
                for img_file in subdir.glob("*.jpg"):
                    shutil.copy(img_file, images_dir / img_file.name)
            for subdir in extract_dir.rglob("labels"):
                for lbl_file in subdir.glob("*.txt"):
                    shutil.copy(lbl_file, labels_dir / lbl_file.name)

    # zipを展開・統合
    extract_and_copy(base_zip)
    extract_and_copy(new_zip)

    # train/val ディレクトリ作成
    for split in ["train", "val"]:
        (images_dir / split).mkdir(parents=True, exist_ok=True)
        (labels_dir / split).mkdir(parents=True, exist_ok=True)

    # train/val 分割
    all_imgs = [f for f in os.listdir(images_dir) if f.endswith(".jpg")]
    random.shuffle(all_imgs)
    n_train = int(len(all_imgs) * split_ratio)
    train_imgs = all_imgs[:n_train]
    val_imgs = all_imgs[n_train:]

    for img_file in train_imgs:
        base_name = Path(img_file).stem
        lbl_file = base_name + ".txt"
        shutil.move(images_dir / img_file, images_dir / "train" / img_file)
        shutil.move(labels_dir / lbl_file, labels_dir / "train" / lbl_file)

    for img_file in val_imgs:
        base_name = Path(img_file).stem
        lbl_file = base_name + ".txt"
        shutil.move(images_dir / img_file, images_dir / "val" / img_file)
        shutil.move(labels_dir / lbl_file, labels_dir / "val" / lbl_file)

    # dataset.yaml 生成
    dataset_yaml = {
        "path": str(merged_dir),
        "train": "images/train",
        "val": "images/val",
        "test": "",
        "names": {0: "Child", 1: "Adult"},
    }

    yaml_path = merged_dir / "dataset.yaml"
    with open(yaml_path, "w") as f:
        yaml.dump(dataset_yaml, f, sort_keys=False)

    # 一時ディレクトリ削除
    shutil.rmtree(tmp_dir)

    print(f"✅ 完了: {merged_dir}")
    print(f"📊 train: {len(train_imgs)}, val: {len(val_imgs)}")
    print(f"📄 dataset.yaml: {yaml_path}")

In [ ]:
merge_and_split_finetune_data(
    base_zip="finetune_base_subset.zip",
    new_zip="okinawa_202510.zip", # ここを差し替える
    output_root="okinawa_202510_for_finetune" # ここを差し替える
)

In [ ]:
from ultralytics import YOLO

In [ ]:

# google drive共有から取ってくる
best_model = YOLO("20250924_best.pt")

In [ ]:
# Train the model
best_model.train(data="/content/okinawa_202510_for_finetune/dataset.yaml", epochs=40, lr0=0.0005, imgsz=640)


In [ ]:
from IPython.display import Image

Image(filename="runs/detect/train/results.png", width=800)